In [1]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder.appName("CreateTable").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 18:13:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 🌊🐳🧊 Example of creating an Iceberg Table

In [7]:
%%sql
CREATE TABLE IF NOT EXISTS iceberg.github_issues (
    actor STRING,
    created_at TIMESTAMP,
    event_id STRING,
    org STRING,
    action STRING,
    issue STRING
) USING ICEBERG
PARTITIONED BY (days(created_at));

++
||
++
++

In [10]:
%%sql
DESCRIBE TABLE iceberg.github_issues;

col_name,data_type,comment
actor,string,None
created_at,timestamp,None
event_id,string,None
org,string,None
action,string,None
issue,string,None
,,
# Partitioning,,
Part 0,days(created_at),


In [15]:
%%sql
DESCRIBE FORMATTED iceberg.github_issues;

col_name,data_type,comment
actor,string,None
created_at,timestamp,None
event_id,string,None
org,string,None
action,string,None
issue,string,None
,,
# Partitioning,,
Part 0,days(created_at),
,,


In [28]:
%%sql
CREATE TABLE IF NOT EXISTS iceberg.dim_repo (
    id BIGINT,
    name STRING,
    url STRING
) USING ICEBERG

++
||
++
++

In [29]:
%%sql
CREATE TABLE IF NOT EXISTS iceberg.dim_org (
    id BIGINT,
    name STRING,
    url STRING
) USING ICEBERG

++
||
++
++

In [30]:
%%sql
CREATE TABLE IF NOT EXISTS iceberg.fct_issue_events (
    event_id BIGINT,       -- Unique event identifier (PK)
    action STRING,         -- Type of action (opened, closed, reopened)
    issue_id BIGINT,       -- FK to Issues dimension
    user_id BIGINT,        -- FK to Users dimension
    org_id BIGINT,         -- FK to Organizations dimension
    created_at TIMESTAMP   -- When the event occurred
) USING ICEBERG
PARTITIONED BY (action);

++
||
++
++